# Docker & Containerization Tutorial

## AI Research Assistant - Containerized Deployment

This tutorial teaches you how to containerize a full-stack application with:
- **Backend**: Python FastAPI + LangGraph
- **Frontend**: React + Vite + Tailwind

---

## 1. What is Docker?

Docker is a platform that packages applications into **containers** - lightweight, standalone units that include everything needed to run:

| Component | Description |
|-----------|-------------|
| Code | Your application source code |
| Runtime | Python, Node.js, etc. |
| Libraries | Dependencies (pip, npm packages) |
| Environment | Environment variables, configs |

### Why Containers?

**Problem**: "It works on my machine" 😤

**Solution**: Package the entire environment, so it runs the same everywhere.

```
┌─────────────────────────────────────────┐
│           Your Laptop                   │
│  ┌─────────────┐  ┌─────────────┐      │
│  │  Backend    │  │  Frontend   │      │
│  │  Container  │  │  Container  │      │
│  │  (Python)   │  │  (Node)     │      │
│  └─────────────┘  └─────────────┘      │
│           Docker Engine                 │
└─────────────────────────────────────────┘
```

## 2. Key Docker Concepts

### Image vs Container

| Concept | Analogy | Description |
|---------|---------|-------------|
| **Image** | Recipe/Blueprint | Read-only template with instructions |
| **Container** | Cake/Building | Running instance of an image |

You can create **many containers** from **one image**.

### Dockerfile

A text file with instructions to build an image:

```dockerfile
FROM python:3.11      # Start from base image
COPY . /app           # Copy files into image
RUN pip install ...   # Run commands during build
CMD ["python", ...]   # Command to run when container starts
```

### Docker Compose

Tool for defining **multi-container** applications in one YAML file.

## 3. Our Project Structure

We'll create this structure:

```
react_fastapi_class/
├── backend/
│   ├── Dockerfile          # Backend container instructions
│   ├── app/
│   ├── requirements.txt
│   └── .env
├── frontend/
│   ├── Dockerfile          # Frontend container instructions
│   ├── src/
│   └── package.json
├── docker-compose.yml      # Orchestrates both containers
└── .env.example            # Template for environment variables
```

---

# Part 1: Backend Dockerfile

---

## 4. Backend Dockerfile Explained

Create this file at `backend/Dockerfile`:

In [ ]:
# backend/Dockerfile

# Stage 1: Use official Python image as base
FROM python:3.11-slim

# Set working directory inside container
WORKDIR /app

# Copy requirements first (for better caching)
COPY requirements.txt .

# Install Python dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy the rest of the application
COPY . .

# Expose port 8000
EXPOSE 8000

# Command to run the application
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"]

### Line-by-Line Breakdown

| Line | Purpose |
|------|---------|
| `FROM python:3.11-slim` | Start from official Python image (`slim` = smaller size) |
| `WORKDIR /app` | Create and switch to `/app` directory |
| `COPY requirements.txt .` | Copy requirements file first |
| `RUN pip install ...` | Install dependencies (cached if requirements unchanged) |
| `COPY . .` | Copy all source code |
| `EXPOSE 8000` | Document that container listens on port 8000 |
| `CMD [...]` | Default command when container starts |

### Why copy requirements.txt separately?

**Docker Layer Caching**: Each instruction creates a layer. If a layer hasn't changed, Docker reuses the cached version.

```
┌─────────────────────────┐
│ COPY . .                │  ← Changes often (your code)
├─────────────────────────┤
│ RUN pip install         │  ← Cached if requirements unchanged
├─────────────────────────┤
│ COPY requirements.txt   │  ← Rarely changes
├─────────────────────────┤
│ FROM python:3.11-slim   │  ← Base layer
└─────────────────────────┘
```

## 5. Backend .dockerignore

Create `backend/.dockerignore` to exclude unnecessary files:

In [ ]:
# backend/.dockerignore

__pycache__
*.pyc
*.pyo
venv/
.env
.git
*.md
.pytest_cache

---

# Part 2: Frontend Dockerfile

---

## 6. Frontend Dockerfile (Development)

For **development**, we use Vite's dev server with hot reload.

Create `frontend/Dockerfile`:

In [ ]:
# frontend/Dockerfile

# Use official Node.js image
FROM node:20-alpine

# Set working directory
WORKDIR /app

# Copy package files first (for caching)
COPY package*.json ./

# Install dependencies
RUN npm install

# Copy source code
COPY . .

# Expose Vite's default port
EXPOSE 5173

# Start development server
CMD ["npm", "run", "dev", "--", "--host", "0.0.0.0"]

### Key Points

| Aspect | Explanation |
|--------|-------------|
| `node:20-alpine` | Alpine = minimal Linux (~50MB vs ~300MB) |
| `--host 0.0.0.0` | Required! Makes Vite accessible outside container |
| `EXPOSE 5173` | Vite's default development port |

## 7. Frontend .dockerignore

Create `frontend/.dockerignore`:

In [ ]:
# frontend/.dockerignore

node_modules
dist
.git
*.md
.env*

⚠️ **Important**: Always ignore `node_modules`! It's large and OS-specific. Let the container install fresh dependencies.

---

# Part 3: Docker Compose

---

## 8. Docker Compose File

Docker Compose orchestrates multiple containers. Create `docker-compose.yml` in the project root:

In [ ]:
# docker-compose.yml

version: '3.8'

services:
  # Backend Service
  backend:
    build: ./backend
    ports:
      - "8000:8000"
    env_file:
      - ./backend/.env
    volumes:
      - ./backend:/app
    command: uvicorn app.main:app --host 0.0.0.0 --port 8000 --reload

  # Frontend Service
  frontend:
    build: ./frontend
    ports:
      - "5173:5173"
    volumes:
      - ./frontend:/app
      - /app/node_modules
    depends_on:
      - backend

### Docker Compose Breakdown

```yaml
services:           # Define our containers
  backend:          # Service name (becomes hostname)
    build: ./backend    # Path to Dockerfile
    ports:
      - "8000:8000"     # host:container port mapping
    env_file:
      - ./backend/.env  # Load environment variables
    volumes:
      - ./backend:/app  # Mount local code → enables hot reload
    command: ...        # Override Dockerfile CMD (adds --reload)
```

### Volume Mount Explained

```
volumes:
  - ./backend:/app      # Sync local files to container
  - /app/node_modules   # EXCEPT node_modules (use container's version)
```

This enables **hot reload** - edit code locally, see changes instantly in container!

## 9. Service Communication

Docker Compose creates a **network** where services can talk to each other by name:

```
┌────────────────────────────────────────────────┐
│              Docker Network                    │
│                                                │
│   ┌──────────────┐      ┌──────────────┐      │
│   │   frontend   │ ───► │   backend    │      │
│   │   :5173      │      │   :8000      │      │
│   └──────────────┘      └──────────────┘      │
│                                                │
│   frontend can reach backend at:               │
│   http://backend:8000                          │
└────────────────────────────────────────────────┘
          │                     │
          ▼                     ▼
   localhost:5173        localhost:8000
   (your browser)        (API access)
```

---

# Part 4: Running the Containers

---

## 10. Essential Docker Commands

Run these commands in your terminal from the project root:

In [ ]:
# Build and start all services
docker-compose up --build

# Run in background (detached mode)
docker-compose up -d --build

# View running containers
docker-compose ps

# View logs
docker-compose logs -f           # All services
docker-compose logs -f backend   # Specific service

# Stop all services
docker-compose down

# Stop and remove volumes (clean slate)
docker-compose down -v

### Command Reference

| Command | Purpose |
|---------|---------||
| `docker-compose up` | Start services |
| `--build` | Rebuild images before starting |
| `-d` | Detached mode (run in background) |
| `docker-compose down` | Stop and remove containers |
| `docker-compose logs -f` | Follow log output |
| `docker-compose exec backend bash` | Open shell in running container |

## 11. Testing Your Setup

After running `docker-compose up --build`:

1. **Backend Health Check**:
   ```
   curl http://localhost:8000/health
   # Expected: {"status": "healthy"}
   ```

2. **Frontend**:
   - Open http://localhost:5173 in browser
   - You should see the chat interface

3. **Test the Chat**:
   - Type a message and verify you get a response
   - Check `docker-compose logs -f backend` for API activity

---

# Part 5: Troubleshooting

---

## 12. Common Issues & Solutions

### Issue: Port already in use
```
Error: Bind for 0.0.0.0:8000 failed: port is already allocated
```
**Solution**: Stop the other process or change the port in docker-compose.yml:
```yaml
ports:
  - "8001:8000"  # Use 8001 on host
```

---

### Issue: Module not found in container
```
ModuleNotFoundError: No module named 'fastapi'
```
**Solution**: Rebuild the image:
```bash
docker-compose build --no-cache backend
```

---

### Issue: Frontend can't connect to backend
```
CORS error or connection refused
```
**Solution**: Ensure you're using `localhost:8000` (not `backend:8000`) from the browser. The service name only works inside the Docker network.

---

### Issue: Changes not reflecting
**Solution**: 
1. Check volume mounts are correct
2. For Python, ensure `--reload` flag is present
3. Try: `docker-compose restart`

## 13. Debugging Commands

In [ ]:
# Open a shell inside a running container
docker-compose exec backend bash
docker-compose exec frontend sh   # Alpine uses sh, not bash

# Check if files are mounted correctly
docker-compose exec backend ls -la /app

# Check environment variables
docker-compose exec backend env | grep OPENAI

# View detailed container info
docker inspect <container_id>

# Check container resource usage
docker stats

---

# Part 6: Production Considerations (Bonus)

---

## 14. Production Frontend (Multi-Stage Build)

For production, we build static files and serve with nginx:

In [ ]:
# frontend/Dockerfile.prod

# Stage 1: Build
FROM node:20-alpine AS builder
WORKDIR /app
COPY package*.json ./
RUN npm ci
COPY . .
RUN npm run build

# Stage 2: Serve with nginx
FROM nginx:alpine
COPY --from=builder /app/dist /usr/share/nginx/html
EXPOSE 80
CMD ["nginx", "-g", "daemon off;"]

### Multi-Stage Build Benefits

| Stage | Image Size | Contents |
|-------|------------|----------|
| Builder | ~400MB | Node.js, npm, source code |
| Final | ~25MB | nginx + static files only |

The final image is **much smaller** because it doesn't include Node.js or source code!

---

# Exercises

---

## Exercise 1: Build and Run

1. Create all the files from this tutorial
2. Run `docker-compose up --build`
3. Test the application at http://localhost:5173
4. Check the logs for both services

## Exercise 2: Modify and Observe Hot Reload

1. With containers running, edit `frontend/src/App.jsx`
2. Change the header text
3. Observe the browser updates automatically

## Exercise 3: Add a Health Check

Add a healthcheck to the backend service in docker-compose.yml:

```yaml
backend:
  # ... existing config ...
  healthcheck:
    test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
    interval: 30s
    timeout: 10s
    retries: 3
```

## Exercise 4: Scale the Backend

Try running multiple backend instances:
```bash
docker-compose up --scale backend=3
```
Note: This requires removing the fixed port mapping first.

---

# Summary

---

## Files Created

| File | Purpose |
|------|---------||
| `backend/Dockerfile` | Build instructions for FastAPI container |
| `backend/.dockerignore` | Files to exclude from backend image |
| `frontend/Dockerfile` | Build instructions for React container |
| `frontend/.dockerignore` | Files to exclude from frontend image |
| `docker-compose.yml` | Orchestrates both services |

## Key Commands

```bash
docker-compose up --build      # Start everything
docker-compose down            # Stop everything
docker-compose logs -f         # View logs
docker-compose exec <svc> sh   # Shell into container
```

## What You Learned

✅ Docker images vs containers  
✅ Writing Dockerfiles for Python and Node.js  
✅ Layer caching for faster builds  
✅ Docker Compose for multi-container apps  
✅ Volume mounts for hot reload  
✅ Service networking  
✅ Multi-stage builds for production  